# Lecture 6: Batch Normalization and Residual Streams

In this lecture, we will discuss two important techniques that have been shown to be very effective in training deep neural networks: Batch Normalization and Residual Streams. We will discuss both of these techniques in detail and show how they can be used to improve the performance of deep neural networks.

### Importing libraries

### Configuration

### Reproducibility

### Device

### Dataset

### Tokenizer

### Preprocessing

### Model

### Training